In [ ]:
import pywikibot
from pywikibot import pagegenerators
from tqdm import tqdm
import re


lang_wiki = pywikibot.Site(code='en', fam='hitchwiki')
if not lang_wiki.user():
    lang_wiki.login()

In [ ]:
places = []
gen = pagegenerators.AllpagesPageGenerator(site=lang_wiki)
for page in gen:
    if any(s in page.text for s in ["{{Infobox"]):
        places.append(page.title())

In [ ]:
len(places)

In [ ]:
format = "''xxx''' {{Coords|lat|lon|1}}"

In [ ]:
https://wego.here.com/portugal/marco-de-canaveses/petrol-station/repsol--620ez64q-b1bb2e884140423cab7e9393743fd2f4?x=ep&map=41.18695,-8.15549,15,normal

In [ ]:
links = []

for place in tqdm(places):
    page = pywikibot.Page(lang_wiki, place)

    pattern = r"(https://osm\.org/go/[^\s\]\)]+|https://www\.openstreetmap\.org/[^\s\]\)]+|https://www\.openstreetmap\.org/node/[^\s\]\)]+|https://goo\.gl/maps/[^\s\]\)]+|https://hitchmap\.com/[^\s\]\)]+|http://maps\.google\.be/maps[^\s\]\)]+|http://maps\.google\.com/maps[^\s\]\)]+|https://www\.google\.co\.uk/maps/[^\s\]\)]+|https://wego\.here\.com/[^\s\]\)]+)"
    matches = re.findall(pattern, page.text)

    if matches:
        t = (place, matches)
        print(t)
        links.append(t)
        

    # page = pywikibot.Page(en_wiki, eng_page_name)
    # page.text = f"{{{{ai-enhanced}}}}\n{translated_page}"
    # page.save(f"Create this article by translation with language model (gpt 4.1) from hitchwiki original language: {lang} {ru_page_name}")
    # print(page.title())
    # print("******")

In [ ]:
locations = []

for link in tqdm(links):
    place, matches = link
    this_locations = []

    pattern = r'(-?\d+\.\d{4,}),(-?\d+\.\d{4,})'


    for match in matches:
        coords = re.search(pattern, match)
        if coords:
            lat, lon = coords.groups()
            this_locations.append((match, (lat, lon)))

    if this_locations:
        locations.append((place, this_locations))

In [ ]:
locations